In [2]:
races =   ['kattekoers'
          ,'giro-ciclistico-d-italia'
          ,'circuito-belvedere'
          ,'gp-di-poggiana'
          ,'gp-industria-e-commercio2'
          ,'tour-des-flandres-espoirs'
          ,'liege-bastogne-liege-u23'
          ,'tour-de-l-avenir'
          ,'la-cote-picarde-nations-cup'
          ,'coppa-della-pace-trofeo-flli-anelli'
          ,'gp-palio-del-recioto'
          ,'giro-ciclistico-della-valle-d-aosta-mont-blanc'

          ,'gp-capodarco'
          ,'gran-premio-della-liberazione'
          ,'gp-industrie-del-marmo'
          ,'gp-di-poggiana'
          ,'piccolo-giro-di-lombardia'
          ,'le-triptyque-des-monts-et-chateaux'
          ,'paris-roubaix-espoirs'
          ,'paris-tours-u23'
          ,'ronde-de-l-isard'
          ,'ruota-d-oro-gp-festa-del-perdono'
          ,'gp-industria-e-commercio2'
          ,'trofeo-piva'
          ,'european-championships-itt-u23'
          ,'european-championships'
          ,'world-championships-itt-u23'
          ,'world-championships-u23'

          #Junior Races
          ,'omloop-der-vlaamse-gewesten2'
          ,'tour-du-valromey'
          ,'liege-la-gleize'
          ,'bernaudeau-junior'
          ,'course-de-la-paix-junior'
          ,'gp-general-patton'
          ,'grand-prix-ruebliland'
          ,'gp-dell-arno'
          ,'keizer-der-juniores'
          ,'la-coupe-du-president-de-la-ville-de-grudziadz'
          ,'trofeo-karlsberg'
          ,'omloop-der-vlaamse-gewesten2'
          ,'paris-roubaix-juniors'
          ,'ronde-van-vlaanderen-juniores'
          ,'sint-martinusprijs-kontich'
          ,'driedaagse-van-axel'
          ,'tour-de-l-abitibi'
          ,'tour-du-pays-de-vaud'
          ,'trofeo-buffoni'
          ,'trofeo-comune-di-vertova'
          ,'trofeo-emilio-paganessi'
          ,'le-trophee-centre-morbihan'
          ,'chrono-des-nations-les-herbiers-vendee-mj'
          ,'european-championship-mj'
          ,'giro-internazionale-della-lunigiana'
          ,'uci-world-championships-itt-mj'
          ,'uci-world-championships-mj'
          ,'int-junioren-rundfahrt-niedersachsen'


          #2.2 Races
          ,'olympias-tour'
          ,'tour-alsace'
          ,'tour-de-normadie'
          ,'eschborn-frankfurt-u23'
          ,'zlm-tour'
          ,'tour-de-berlin'
          ,'paris-arras-tour'
          ,'tour-des-pays-de-savoie']

years = range(2007,2023)

In [3]:
#Importing packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import pyarrow.parquet as pq
import time
import os
from bs4 import BeautifulSoup

#Initiate driver
DRIVERPATH = "C:/Users/banjanss/AppData/Local/Google/Chrome/Application/chromedriver.exe"
#Path where your google chrome has been installed
CHROMEPATH= "C:/Users/banjanss/AppData/Local/Google/Chrome/Application/chrome.exe"
G_options = webdriver.ChromeOptions()
G_options.binary_location = CHROMEPATH
driver = webdriver.Chrome(DRIVERPATH,options= G_options)

C:\Users\banjanss\AppData\Local\Temp\1\ipykernel_2244\2254019055.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVERPATH,options= G_options)


In [1]:
import numpy as np

def name_adjustment(test_df):
    try:
        test_df['Team'] = np.where(test_df['Team'].isna(), '', test_df['Team'])
        test_df['ActualName'] = [a.replace(b, '').strip() for a, b in zip(test_df['Rider'], test_df['Team'])]
        return(test_df)
    except:            
        return('Bad DataFrame')

results_list = []

for race in races:
    for year in years:
        driver.get('https://www.procyclingstats.com/race/'+race+'/'+str(year))
        if 'Page not found' in driver.title:
            print('Race not held')
            print(race)
            continue
        soup = BeautifulSoup(driver.page_source, 'lxml')
        tables = soup.find_all('table')
        dfs = pd.read_html(str(tables))
        if ' Stage ' in driver.title:
            print('Stage Race')
            print(race)
            final_table = dfs[1]
        else:
            print('One Day Race')
            print(race)
            final_table = dfs[0]
        final_table = name_adjustment(final_table)
        try:
            final_table = final_table[['ActualName', 'Rnk']]
        except:
            print('Data Quality Issue')
            continue
        final_table['Race'] = race
        final_table['Year'] = year
        results_list.append(final_table)           
                   

NameError: name 'races' is not defined

In [32]:
full_df = pd.concat(results_list)

In [33]:
full_df

,ActualName,Rnk,Race,Year
0,DELFOSSE Sébastien,1,kattekoers,2007
1,VAN DER SCHUEREN Kenny,2,kattekoers,2007
2,VANSPEYBROUCK Pieter,3,kattekoers,2007
3,ROELANDTS Jürgen,4,kattekoers,2007
4,EECKHOUT Kevin,5,kattekoers,2007
...,...,...,...,...
67,RODRIGUEZ Jordan Santiago,68,tour-des-pays-de-savoie,2021
68,LA GRENADE Jérémie,69,tour-des-pays-de-savoie,2021
69,RIVARD Nicolas,70,tour-des-pays-de-savoie,2021
70,CARREAU Lukas,71,tour-des-pays-de-savoie,2021


In [34]:
full_df.to_csv('AllYouthRaceResults.csv')

In [1]:
import pandas as pd

full_df = pd.read_csv('AllYouthRaceResults.csv')

In [2]:
import numpy as np

#Reset values of DNFs
full_df['Rnk'] = np.where(full_df['Rnk']=='DNF', 249, full_df['Rnk'])
full_df['Rnk'] = np.where(full_df['Rnk']=='DNS', 249, full_df['Rnk'])
full_df['Rnk'] = np.where(full_df['Rnk']=='DSQ', 249, full_df['Rnk'])
full_df['Rnk'] = np.where(full_df['Rnk']=='OTL', 249, full_df['Rnk'])
full_df['Rnk'] = full_df['Rnk'].astype(int)

In [3]:
#Inverse weights: low rank should be given high weight

full_df['Rnk'] = 250 - full_df['Rnk']

In [4]:
edge_list = full_df.groupby(['ActualName', 'Race'])['Rnk'].max()
edge_list = pd.DataFrame(edge_list)
edge_list = edge_list.reset_index()
edge_list_tuples = list(edge_list.itertuples(index=False, name=None))

In [5]:
edge_list_tuples

[('AADEL Reda', 'world-championships-itt-u23', 193),
 ('AADEL Reda', 'world-championships-u23', 190),
 ('AAEN JØRGENSEN Jonas', 'la-cote-picarde-nations-cup', 229),
 ('AAEN JØRGENSEN Jonas', 'le-triptyque-des-monts-et-chateaux', 244),
 ('AAEN JØRGENSEN Jonas', 'liege-bastogne-liege-u23', 225),
 ('AAEN JØRGENSEN Jonas', 'olympias-tour', 223),
 ('AAEN JØRGENSEN Jonas', 'tour-de-l-avenir', 175),
 ('AAEN JØRGENSEN Jonas', 'tour-de-normadie', 245),
 ('AAEN JØRGENSEN Jonas', 'world-championships-u23', 244),
 ('AAEN JØRGENSEN Jonas', 'zlm-tour', 237),
 ('AAGAARD HANSEN Jonathan', 'keizer-der-juniores', 169),
 ('AAGAARD HANSEN Jonathan', 'sint-martinusprijs-kontich', 220),
 ('AAGAARD HANSEN Tobias', 'european-championship-mj', 191),
 ('AAGAARD HANSEN Tobias', 'grand-prix-ruebliland', 180),
 ('AAGAARD HANSEN Tobias', 'trofeo-karlsberg', 177),
 ('AAKVIK Niklas', 'la-cote-picarde-nations-cup', 166),
 ('AAKVIK Niklas', 'liege-bastogne-liege-u23', 202),
 ('AAKVIK Niklas', 'olympias-tour', 228),
 ('

In [6]:
#Make sure networkx and nodevectors are compatible with already installed packages!

from nodevectors import Node2Vec
import networkx as nx

G = nx.Graph()  
G.add_nodes_from(set(edge_list['ActualName']), type='Rider')
G.add_nodes_from(set(edge_list['Race']), type='Race')
G.add_weighted_edges_from(edge_list_tuples)

In [7]:
#Overwrite function to unsure compatibility
nx.adj_matrix = nx.adjacency_matrix

In [8]:
g2v = Node2Vec(n_components=32,
            walklen = 10,
            epochs = 20)
g2v.fit(G)

Making walks... Done, T=9.04
Mapping Walk Names... Done, T=2.83
Training W2V... WARNING: gensim word2vec version is unoptimizedTry version 3.6 if on windows, versions 3.7 and 3.8 have had issues


C:\Users\banjanss\AppData\Roaming\Python\Python39\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


Done, T=61446.28


In [14]:
g2v.model.wv['ALBANESE Vincenzo']

array([-1.4887363 ,  2.0106645 , -2.4336019 ,  1.1858972 , -1.5524663 ,
       -0.56646687,  2.9552565 , -0.59410137,  1.9378501 , -3.091099  ,
       -0.77254206,  1.3293971 ,  0.2787965 , -0.2887829 ,  0.40321583,
        0.4384794 , -1.5435672 ,  1.446928  , -0.4828393 , -0.71136016,
        0.19824201, -1.7620307 , -1.3949089 , -0.1179001 , -0.10895688,
        0.42832452, -0.10169653, -0.5819642 ,  0.28247994, -1.7488301 ,
        1.5959488 ,  0.5846482 ], dtype=float32)

In [27]:
#Retrieve vectors
rider_names = list(dict.fromkeys([el[0] for el in edge_list_tuples]))
vectors = []

for rider in rider_names:
    vectors.append(g2v.model.wv[rider])
    
embeddings = pd.DataFrame(vectors)
embeddings['Rider_Name'] = rider_names

In [41]:
# Little toy example
# Riders do not seem very similar besides race program
# Probably other weighting method needed or other walking algorithm (i.e., weight plays role in walking)
# I also believe the current list of races to be very limited (e.g. Julian Alaphilippe only has results in 5/60)
def output_most_similar_rider(name):
    all_other_riders = embeddings[embeddings['Rider_Name']!=name]
    diff_df = all_other_riders.drop('Rider_Name', axis = 1) - g2v.model.wv[name]
    norm_df = diff_df.apply(np.linalg.norm, axis=1)
    print(all_other_riders.loc[norm_df.idxmin()]['Rider_Name'])
    
output_most_similar_rider('PIDCOCK Thomas')
output_most_similar_rider('ALMEIDA João')    

BARAĆ Antonio
CARVALHO Andre


In [40]:
embeddings.head(5)

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,Rider_Name
0,0.429872,0.654289,0.288489,-0.494263,0.153994,0.759591,0.843510,-0.105606,-0.287398,-0.599084,...,-1.329375,-0.833920,0.092224,-0.222338,-1.183740,-0.626180,-2.008351,1.393554,0.408902,AADEL Reda
1,-1.102521,-0.072375,1.535035,-0.499040,0.475663,-0.888514,0.395978,0.949461,-0.060149,-0.416326,...,-2.470551,-0.042328,0.333598,-1.913737,-1.354953,-1.208918,-3.383505,-0.008691,0.360961,AAEN JØRGENSEN Jonas
2,-0.730316,-0.895771,1.098583,0.322986,0.733673,-0.905697,-0.554789,0.932219,-1.327662,1.723725,...,-0.901462,1.733434,-0.828098,0.797557,-1.039755,0.550194,-0.280529,-1.356885,-1.374084,AAGAARD HANSEN Jonathan
3,-0.345012,0.304171,0.587456,1.672497,-0.523137,0.304716,-0.030343,-0.633966,-0.908220,2.040785,...,-0.153756,-0.767591,1.008583,1.782456,-1.471497,-0.303125,-0.950999,0.876305,-0.440363,AAGAARD HANSEN Tobias
4,-0.000380,0.558321,1.106088,-0.525072,1.127038,-0.635184,-0.641364,0.766005,0.211975,-0.408999,...,-2.410130,-0.307845,-0.940301,-1.371124,-0.975683,-1.608279,-1.406225,0.029710,-0.165549,AAKVIK Niklas


In [46]:
import pickle

with open('FirstFittedNode2Vec.pickle', 'wb') as f:
    pickle.dump(g2v, f)
    
embeddings.to_csv('EmbeddingsFromFirstMethod.csv')

In [45]:
# Load fitted skip-gram model

import pickle

with open('FirstFittedNode2Vec.pickle', 'rb') as f:
    loaded_vectors = pickle.load(f)

loaded_vectors.model.wv['BARAĆ Antonio']

array([-1.6069225 ,  1.9766173 ,  0.29634213, -0.67392844, -0.10007078,
        0.57182276,  1.7460359 , -1.2578045 , -0.741277  , -1.6114342 ,
        1.6719358 , -0.91841185, -0.1186644 ,  1.1885985 , -2.0599463 ,
       -1.0154957 ,  1.5014405 , -0.96993923, -1.0305482 ,  1.4222978 ,
       -0.5097695 , -1.388363  ,  1.0303653 , -1.5459269 , -0.7785537 ,
        0.0356098 , -1.222704  , -2.7964077 , -0.17281803, -0.8355371 ,
        0.64234054,  1.0491314 ], dtype=float32)